In [ ]:
pip install causalml scikit-learn matplotlib


In [ ]:
import pandas as pd
import numpy as np

In [7]:
from causalml.match import NearestNeighborMatch
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [28]:
from causalml.metrics import *

In [69]:
df = pd.read_csv("data2.csv")

In [71]:
#encoding gender column
df['gender'] = df['gender'].map({'male': 1, 'female': 0})

In [72]:
#cleaning if their is any missing values
for col in df.columns:
    if df[col].isnull().any():
        if not df[col].dropna().empty: 
            if df[col].dtype == 'object':
                df[col].fillna(df[col].mode()[0], inplace=True)
            else:
                df[col].fillna(df[col].mean(), inplace=True)

In [76]:
#normalizing age
scaler = StandardScaler()
df['age'] = scaler.fit_transform(df[['age']])

In [78]:
X = df[['age', 'gender']]    
treatment = df['treatment']   
y = df['outcome']  

In [79]:
#logical regretion for calculating psm score with the cofactors age and gender
model = LogisticRegression()
model.fit(X, treatment)
propensity_scores = model.predict_proba(X)[:, 1]

In [80]:
df['propensity_score'] = propensity_scores

In [ ]:
#maching treated and controled with the psm score nearest  
matcher = NearestNeighborMatch(replace=False, ratio=1)
matched = matcher.match(
    data=df,
    treatment_col='treatment',
    score_cols=['propensity_score']
)